In [1]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# if using a Jupyter notebook, includue:
%matplotlib inline

In [2]:
df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')

"""
looking through all valid spacings (if it si invalid it goes below 0)
and displaying minimal maximal and rounded mean spacing and median
in my case median and mean values are close - and using the median values will lead to a bit less interpolations later
"""
spacingDict={}
for keyWord in ['t2w','adc', 'cor','hbv','sag'  ]: 
    for addedKey in ['_spac_x','_spac_y','_spac_z']:   
        colName = keyWord+addedKey
        liist = list(filter(lambda it: it>0 ,df[colName].to_numpy() ))
        minn=np.min(liist)                
        maxx=np.max(liist)
        meanRounded = round((minn+maxx)/2,1)
        medianRounded = round(np.median(liist),1)
        spacingDict[colName]=(minn,maxx,meanRounded,medianRounded)

spacingDict



{'t2w_spac_x': (0.234375, 0.78125, 0.5, 0.5),
 't2w_spac_y': (0.234375, 0.78125, 0.5, 0.5),
 't2w_spac_z': (2.200000060372773, 5.000000066297424, 3.6, 3.0),
 'adc_spac_x': (0.859375, 2.5098040103912, 1.7, 2.0),
 'adc_spac_y': (0.859375, 2.5098040103912, 1.7, 2.0),
 'adc_spac_z': (2.999998832062665, 5.000000232855622, 4.0, 3.0),
 'cor_spac_x': (0.286756128073, 0.9765625, 0.6, 0.6),
 'cor_spac_y': (0.286756128073, 0.9765625, 0.6, 0.6),
 'cor_spac_z': (0.9999999999999636, 4.49999998188102, 2.7, 3.0),
 'hbv_spac_x': (0.859375, 2.5098040103912, 1.7, 2.0),
 'hbv_spac_y': (0.859375, 2.5098040103912, 1.7, 2.0),
 'hbv_spac_z': (2.999998832062665, 5.000000232855622, 4.0, 3.0),
 'sag_spac_x': (0.254629641771, 0.78125, 0.5, 0.6),
 'sag_spac_y': (0.254629641771, 0.78125, 0.5, 0.6),
 'sag_spac_z': (2.999999882111563, 4.000404550206146, 3.5, 3.6)}

In [11]:
from KevinSR import mask_interpolation, SOUP_GAN
import os
import numpy as np
from scipy import ndimage, interpolate
from scipy.ndimage import zoom
from KevinSR import SOUP_GAN
import monai
import SimpleITK as sitk
from copy import deepcopy


def superSampleinZ(data, currentSpacZ, targetSpacZ):
    """
    perform supersampling in Z direction
    currentSpac, targetSpac - current and target z spacing - float numbers
    """
    Z_FAC = currentSpacZ/targetSpacZ
    # Call the SR interpolation tool from KevinSR
    thins_gen = SOUP_GAN(data, Z_FAC, 0)
    return thins_gen
    
def subsample(image, targetSpacZ):
    """
    perform subsampling to targetSpac
    TODO() bicubic interpolation adapted from 
        https://itk.org/Doxygen/html/WikiExamples_2ImageProcessing_2Upsampling_8cxx-example.html
        https://discourse.itk.org/t/resample-produces-blurry-results-when-just-cropping/4473/8
    """    
    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(targetSpacZ)
    resample.SetOutputDirection(image.GetDirection())
    resample.SetOutputOrigin(image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(image.GetPixelIDValue())
    resample.SetInterpolator(sitk.sitkBSpline)
    return resample.Execute(image) 


def transformSpacingTo(path, targetSpac):
    """
    transforming spacing of the image which path is supplied to target spacing values
    upsampling will be performed with SOUP-GAN  and downsampling will be done via simple itk bicubic interpolation
    """
    image = sitk.ReadImage(path)
    spac = image.GetSpacing()
    spacGoal=list(deepcopy(spac))    
    
    for axis in [0,1,2]:
        #tuple are immutable ...
        spacGoal[axis]=targetSpac[axis]
        print(f"spac {spac} axis {axis} spacing goal {spacGoal} ")
        if(targetSpac[axis]>spac[axis]):
            #if target spacing is bigger we need to subsample

            #overwriting image by resampled one
            image=subsample(image, tuple(spacGoal))
            pass
        if(targetSpac[axis]<spac[axis]):
            #we need to perform supersampling - now as supersampling can be performed only in z direction
            #we need to rotate image accordingly       
            data = sitk.GetArrayFromImage(image)
            if(axis==1):
                transformed = np.moveaxis(data, 0, 2)
                data=superSampleinZ(transformed, spac[axis], targetSpac[axis])
                #getting back original shape
                data = np.moveaxis(data, 2, 0)
            if(axis==2):
                transformed = np.moveaxis(data, 1, 2)
                data=superSampleinZ(transformed, spac[axis], targetSpac[axis])
                #getting back original shape
                data = np.moveaxis(data, 2, 1)                
            if(axis==3):
                # no need to transform
                data=superSampleinZ(data, spac[axis], spac[axis], targetSpac[axis])
            #recreate the image and ovewrite original
            image1 = sitk.GetImageFromArray(data)
            image1.SetSpacing(tuple(spacGoal) #updating spacing
            image1.SetOrigin(image.GetOrigin())
            image1.SetDirection(image.GetDirection())
            image = image1
            #in case of equality we do nothing
    return image






In [12]:
path ='/home/sliceruser/data/orig/10001/10001_1000001_hbv_bf_corr.mha'
spacc=(0.2,0.2,2.0)

image = sitk.ReadImage(path)

subsample(image, spacc)
imagee=transformSpacingTo(path,spacc)
imagee.GetSpacing()

spac (2.0, 2.0, 3.6000000778885344) axis 0 spacing goal (0.2, 2.0, 3.6000000778885344) 


TypeError: 'tuple' object does not support item assignment

In [7]:
imagee.GetSpacing()

(2.0, 2.0, 2.0)

In [6]:
# import SimpleITK as sitk
# import itk 
# a='/home/sliceruser/data/orig/10029/10029_1000029_adc.mha'
# imageSimple=sitk.ReadImage(a)
# image = itk.GetImageFromArray(sitk.GetArrayFromImage(imageSimple))

# interpolator = itk.BSplineInterpolateImageFunction.New(image)
# interpolator.SetSplineOrder(3)
# transform = itk.IdentityTransform.New()
# resampled_image = itk.resample_image_filter(
#     image, interpolator=interpolator, transform=transform,
#     size=itk.size(image), output_spacing=itk.spacing(image), output_origin=itk.origin(image)
# )
# resampled_grid = itk.GetArrayViewFromImage(resampled_image)

TypeError: in method 'itkResampleImageFilterIUS3IUS3_SetTransform', argument 2 of type 'itkTransformD33 const *'